In [ ]:
# -----------------------------------------------------
# Auhor: Timur Abdualimov, ATP
# Competition: MKB 
# First date code: 10.11.2021
# Used: Jupyter Notebook, GPU
# -----------------------------------------------------

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
from matplotlib.pyplot import figure
import seaborn as sns

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer

In [2]:
train, test = pd.read_csv('train_dataset_hackathon_mkb.csv', sep=';', encoding='cp1251'), pd.read_csv('test_dataset_hackathon_mkb.csv', sep=';', encoding='cp1251')

In [3]:
def param_data(data):
    """dataset required parameters """
    param = pd.DataFrame({
              'dtypes': data.dtypes.values,
              'nunique': data.nunique().values,
              'isna': data.isna().sum().values,
              'loc[0]': data.loc[0].values,
              }, 
             index = data.loc[0].index)
    return param

pd.concat([param_data(train), param_data(test)], 
          axis=1, 
          keys = [f'↓ ОБУЧАЮЩАЯ ВЫБОРКА ↓ {train.shape}', f'↓ ТЕСТОВАЯ ВЫБОРКА ↓ {test.shape}'],  
          sort=False)

↓ ОБУЧАЮЩАЯ ВЫБОРКА ↓ (17891, 124)                 \
                                              dtypes nunique   isna   
id_contract                                    int64   17891      0   
id_client                                      int64    8922      0   
SIGN_DATE                                     object     439      0   
IP_flag                                        int64       2      0   
TARGET                                         int64       2      0   
...                                              ...     ...    ...   
COUNT_CHANGE_EVER                            float64      16  16382   
BIRTHDATE                                     object     844  16472   
AGE                                          float64      54  16472   
SEX_NAME                                      object       2  15723   
CITIZENSHIP_NAME                              object       4  15763   

                                      ↓ ТЕСТОВАЯ ВЫБОРКА ↓ (7330, 123)  \
                               loc[0]                           dtypes   
id_contract                         1                            int64   
id_client                        1847                            int64   
SIGN_DATE          01JAN2018:00:00:00                           object   
IP_flag                             0                            int64   
TARGET                              0                              NaN   
...                               ...                              ...   
COUNT_CHANGE_EVER                 NaN                          float64   
BIRTHDATE                         NaN                           object   
AGE                               NaN                          float64   
SEX_NAME                          NaN                           object   
CITIZENSHIP_NAME                  NaN                           object   

                                                       
                  nunique    isna              loc[0]  
id_contract        7330.0     0.0               17892  
id_client          4761.0     0.0                3620  
SIGN_DATE           248.0     0.0  08APR2019:00:00:00  
IP_flag               2.0     0.0                   1  
TARGET                NaN     NaN                 NaN  
...                   ...     ...                 ...  
COUNT_CHANGE_EVER    13.0  6843.0                 NaN  
BIRTHDATE           485.0  6715.0                 NaN  
AGE                  50.0  6715.0                 NaN  
SEX_NAME              2.0  5896.0                 NaN  
CITIZENSHIP_NAME      4.0  5899.0                 NaN  

[124 rows x 8 columns]

In [4]:
# def viz_na(data):
#     """NA visualisation"""
#     global cols
#     cols = data.columns # запишем названия строки сделаем переменную глобальной
#     # определяем цвета 
#     # желтый - пропущенные данные, синий - не пропущенные
#     colours = ['#30d5c8', '#960018'] 
#     sns.heatmap(data[cols].isnull(), cmap=sns.color_palette(colours))
#     plt.show()


# viz_na(train)
# viz_na(test)

In [5]:
# def stat_na_per_percent(data):
#     print(f'{data.shape}')
#     for col in data.columns:
#         pct_missing = np.mean(data[col].isnull())
#         print('{} - {}%'.format(col, round(pct_missing*100)))
#     print("END", end = '\n\n')
# stat_na_per_percent(train)
# stat_na_per_percent(test)

In [6]:
# train.isna().sum().plot.bar();

In [7]:
# sns.countplot(x=train['TARGET']);

In [8]:
# sns.heatmap(train.corr(), mask=np.triu(np.ones_like(train.corr(), dtype=np.bool)));

In [9]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['TARGET'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = train.append(test, sort=False).reset_index(drop=True) # объединяем

data.drop(['id_contract'], axis=1, inplace=True)

In [10]:
for i in data:
    if np.mean(data[i].isna()) >= 0.4:
        data.drop([i], axis=1, inplace=True)

In [12]:
# viz_na(data)

In [13]:
data.dtypes

id_client                  int64
SIGN_DATE                 object
IP_flag                    int64
TARGET                     int64
F1200                    float64
F1230                    float64
F1250                    float64
F1300                    float64
F1500                    float64
F1520                    float64
F1600                    float64
F1700                    float64
F2110                    float64
F2120                    float64
F2200                    float64
F2300                    float64
F2400                    float64
EGRPOINCLUDED            float64
DATEFIRSTREG              object
OKFS_GROUP                object
OKOPF_GROUP               object
OKOGU_GROUP               object
OKATO_REGIONCODE         float64
OKATO_FED                float64
OKTMO_CODE               float64
OKTMO_FED                float64
WORKERSRANGE              object
TAXREG_REGDATE            object
TAXREGPAY_REGDATE         object
COUNTCOOWNERFCSM         float64
COUNTCOOWN

In [14]:
data['SIGN_DATE'] = pd.to_datetime(data['SIGN_DATE'], format = '%d%b%Y:%H:%M:%S').values.astype(np.int64) // 10 ** 9

In [15]:
data['DATEFIRSTREG'] = pd.to_datetime(data['DATEFIRSTREG'], format = '%d%b%Y:%H:%M:%S').values.astype(np.int64) // 10 ** 9
data['DATEFIRSTREG'] = data['DATEFIRSTREG'].apply(lambda x: 0 if x <=0 else x)

In [16]:
data['OKFS_GROUP'].isna().sum()

5283

In [17]:
dict_OKFS_GROUP = {'Частная собственность':0,
                   'Государственная собственность': 1,
                   'Иностранная собственность':2, 
                   'Смешенная российская собственность':3,
                   'Совместная российская и иностранная собственность': 4}
data['OKFS_GROUP'] = data['OKFS_GROUP'].map(dict_OKFS_GROUP).fillna(0)

In [18]:
data['OKOGU_GROUP'].value_counts()

Группировки хозяйствующих субъектов и общественных объединений    19259
Государственная власть в субъектах РФ                               402
Государственная власть в РФ                                         261
Местное самоуправление в РФ                                          13
Name: OKOGU_GROUP, dtype: int64

In [19]:
data['OKOGU_GROUP'].isna().sum()

5286

In [20]:
dict_OKOGU_GROUP = {'Группировки хозяйствующих субъектов и общественных объединений':0,
                   'Государственная власть в субъектах РФ': 1,
                   'Государственная власть в РФ':2, 
                   'Местное самоуправление в РФ':3}
data['OKOGU_GROUP'] = data['OKOGU_GROUP'].map(dict_OKOGU_GROUP).fillna(0)

In [21]:
data['WORKERSRANGE'].value_counts()

0 .. 5          8809
16 .. 50        2894
6 .. 10         2498
51 .. 100       1086
11 .. 15         960
501 .. 1000      708
1001 .. 5000     699
251 .. 500       640
101 .. 150       377
151 .. 200       269
201 .. 250       206
5000 и более     184
Name: WORKERSRANGE, dtype: int64

In [22]:
dict_WORKERSRANGE = {'0 .. 5': 0,
                     '16 .. 50': 1,
                     '6 .. 10': 2, 
                     '51 .. 100': 3,
                     '11 .. 15': 4,
                     '501 .. 1000': 5,
                     '251 .. 500': 6, 
                     '101 .. 150': 7,
                     '151 .. 200': 8,
                     '201 .. 250': 9,
                     '251 .. 500': 10, 
                     '5000 и более': 11}
                    
                    
                    
data['WORKERSRANGE'] = data['WORKERSRANGE'].map(dict_WORKERSRANGE).fillna(0)

In [23]:
data['TAXREG_REGDATE'] = pd.to_datetime(data['TAXREG_REGDATE'], format = '%d%b%Y:%H:%M:%S').values.astype(np.int64) // 10 ** 9
data['TAXREG_REGDATE'] = data['TAXREG_REGDATE'].apply(lambda x: 0 if x <=0 else x)

In [24]:
data['TAXREGPAY_REGDATE'] = pd.to_datetime(data['TAXREGPAY_REGDATE'], format = '%d%b%Y:%H:%M:%S').values.astype(np.int64) // 10 ** 9
data['TAXREGPAY_REGDATE'] = data['TAXREGPAY_REGDATE'].apply(lambda x: 0 if x <=0 else x)

In [25]:
data['OKVED_CODE'].value_counts()

46.46      2692
41.20       993
46.46.1     779
46.90       610
46.31       574
           ... 
84.11.4       1
52.23.1       1
52.10.3       1
01.49.4       1
47.53         1
Name: OKVED_CODE, Length: 879, dtype: int64

In [26]:
labelencoder = LabelEncoder()
data['OKVED_CODE'] = labelencoder.fit_transform(data['OKVED_CODE'])

In [27]:
data['OKOPF_GROUP'].value_counts()

Коммерческая корпоративная организация      18350
Коммерческая унитарная организация            739
Некоммерческая унитарная организация          585
Некоммерческая корпоративная организация      133
Без прав юридического лица                    131
Name: OKOPF_GROUP, dtype: int64

In [28]:
data['OKOPF_GROUP'].isna().sum()

5283

In [29]:
dict_OKOPF_GROUP = {'Коммерческая корпоративная организация':0,
                   'Коммерческая унитарная организация': 1,
                   'Некоммерческая унитарная организация':2, 
                   'Некоммерческая корпоративная организация':3,
                   'Без прав юридического лица': 4}
data['OKOPF_GROUP'] = data['OKOPF_GROUP'].map(dict_OKOPF_GROUP).fillna(0)

In [30]:
data.dtypes

id_client                  int64
SIGN_DATE                  int64
IP_flag                    int64
TARGET                     int64
F1200                    float64
F1230                    float64
F1250                    float64
F1300                    float64
F1500                    float64
F1520                    float64
F1600                    float64
F1700                    float64
F2110                    float64
F2120                    float64
F2200                    float64
F2300                    float64
F2400                    float64
EGRPOINCLUDED            float64
DATEFIRSTREG               int64
OKFS_GROUP               float64
OKOPF_GROUP              float64
OKOGU_GROUP              float64
OKATO_REGIONCODE         float64
OKATO_FED                float64
OKTMO_CODE               float64
OKTMO_FED                float64
WORKERSRANGE             float64
TAXREG_REGDATE             int64
TAXREGPAY_REGDATE          int64
COUNTCOOWNERFCSM         float64
COUNTCOOWN

In [32]:
# viz_na(data)

In [33]:
imputer = KNNImputer(n_neighbors=3)
Y = data['TARGET']
sample = data['sample']
data.drop(['TARGET', 'sample'], axis = 1, inplace = True)
data = pd.DataFrame(imputer.fit_transform(data), columns = data.columns)
data['TARGET'] = Y.values
data['sample'] = sample.values

In [34]:
data.to_csv('data_prepared_to_AI_2', index=False)

In [377]:
data.isna().sum().sum()

0

In [378]:
data

,id_client,SIGN_DATE,IP_flag,TARGET,DATEFIRSTREG,OKATO_REGIONCODE,OKATO_FED,OKTMO_CODE,OKTMO_FED,TAXREG_REGDATE,TAXREGPAY_REGDATE,OKVED_CODE,sample
0,1847,1514764800,0,0,744940800,3.0,40.0,3.701000e+09,40.0,1151712000,0,255,1
1,4650,1514764800,1,0,0,0.0,0.0,0.000000e+00,0.0,0,0,879,1
2,4770,1514764800,0,0,697161600,33.0,33.0,3.371000e+10,33.0,1030665600,0,274,1
3,12237,1514764800,0,0,1120176000,71.0,34.0,7.187600e+10,34.0,1120176000,0,255,1
4,9988,1514764800,1,0,0,0.0,0.0,0.000000e+00,0.0,0,0,879,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25216,8128,1577664000,0,0,1509062400,63.0,33.0,6.370100e+10,33.0,1509062400,0,755,0
25217,1132,1577664000,0,0,1233446400,92.0,33.0,9.260810e+10,33.0,1233446400,0,255,0
25218,4932,1577750400,0,0,1423094400,37.0,34.0,3.770100e+10,34.0,1423094400,0,309,0
25219,537,1577750400,0,0,1511222400,85.0,40.0,8.570100e+10,40.0,1511222400,0,364,0


In [6]:
# %%html
# <style id=hide>div.input{display:none;}</style>
# <button type="button" 
# onclick="var myStyle = document.getElementById('hide').sheet;myStyle.insertRule('div.input{display:inherit !important;}', 0);">
# Show inputs</button>

In [7]:
#from hideBlock import hide_sloutionution